In [8]:
import tensorflow as tf
import os
tf.get_logger().setLevel('INFO')

2022-10-15 21:35:48.257468: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [10]:
!ls data/model_test_images/

advocate ball     test     train


## Create train/test

In [11]:
!mkdir data/model_test_images/train
!mkdir data/model_test_images/test

mkdir: data/model_test_images/train: File exists
mkdir: data/model_test_images/test: File exists


In [12]:
# Make splits of images


In [36]:
target_dirs = os.listdir("data/model_test_images")
target_dirs

['test', 'ball', 'train', 'advocate']

In [17]:
# train_split = (int(0.8 * len(ball_images)))
# train_split

40

In [19]:
# import numpy as np

# # Get random indexs of certain number from list of images
# train_idx = np.arange(len(ball_images))
# train_idx

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [35]:
import random
def create_train_test_list(target_dir):
    random.seed(42)
    image_list = os.listdir(target_dir)
    train_split = int(0.8 * len(image_list))
    train_image_list = random.sample(image_list,train_split)
    test_image_list = list(set(image_list).difference(set(train_image_list)))
    return train_image_list, test_image_list
train_image_list, test_image_list = create_train_test_list("data/model_test_images/ball")


In [34]:
len(train_image_list), len(test_image_list)

(40, 10)

In [ ]:
# Create a function to move images


In [ ]:
for dirs,sub_dirs,files in os.walk("data/model_test_images/"):
    print(dirs)
    print(sub_dirs)
    print(files)